In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import gc
import torch
import zipfile

In [ ]:
#archivo = zipfile.ZipFile('ml10m.zip')
archivo = zipfile.ZipFile('ml-1m.zip')

In [ ]:
archivo.extractall('')

In [5]:
movie_col = ['movie_id', 'titulo', 'genero']
movie = pd.read_csv('ml-1m/movies.dat', sep='::', names=movie_col)

/home/mich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [6]:
movie.head()

,movie_id,titulo,genero
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# Preprocesamiento de los datos de movie

Vamos a separar el año del titulo para que sea más fácil hacer la búsqueda de los titulos sin necesidad de poner el año de estreno.

In [7]:
nombres = movie['titulo'].tolist() #Sacamos los titulos de las películas en una lista para hacer split entre los nombres y el año

In [8]:
#A cada elemento de nombre se le divide en 2 partes: el nombre de la película dentro de la lista name 
#y dentro de la lista year va el año de cada película
#A cada elemento de nombre se le divide en 2 partes: el nombre de la película dentro de la lista name 
#y dentro de la lista year va el año de cada película
name = []
year = []
for i in nombres:
    name.append(i[:-7])
    year.append(i[-6:])

In [9]:
name[:10]

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye']

In [10]:
year[:10]

['(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)',
 '(1995)']

In [11]:
#Convertimos ambas listas con dataframes para poder agregarlas al df final
name = pd.DataFrame(name, columns=['titulo'])
years = pd.DataFrame(year, columns=['year'])

In [12]:
movie.drop(movie.columns[1], axis=1, inplace=True) # Quitamos la columna de titulo del df original

movie = pd.merge(movie, name, left_on=None, right_on=None, left_index=True, right_index=True)
movie = pd.merge(movie, years, left_on=None, right_on=None, left_index=True, right_index=True)

In [13]:
movie.head()

,movie_id,genero,titulo,year
0,1,Animation|Children's|Comedy,Toy Story,(1995)
1,2,Adventure|Children's|Fantasy,Jumanji,(1995)
2,3,Comedy|Romance,Grumpier Old Men,(1995)
3,4,Comedy|Drama,Waiting to Exhale,(1995)
4,5,Comedy,Father of the Bride Part II,(1995)


In [14]:
cols = ['movie_id', 'titulo', 'year', 'genero'] #Reordenamos las columnas solo para que se vea más bonito
movie = movie[cols]
movie.head()

,movie_id,titulo,year,genero
0,1,Toy Story,(1995),Animation|Children's|Comedy
1,2,Jumanji,(1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men,(1995),Comedy|Romance
3,4,Waiting to Exhale,(1995),Comedy|Drama
4,5,Father of the Bride Part II,(1995),Comedy


In [15]:
rating_col = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=rating_col, usecols=range(3))

/home/mich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [16]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [17]:
usuario_col = ['user_id']
usuarios = pd.read_csv('ml-1m/users.dat', sep='::', names=usuario_col, usecols=range(1))

/home/mich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [18]:
usuarios.head()

,user_id
0,1
1,2
2,3
3,4
4,5


In [19]:
movie_rating = pd.merge(movie, ratings) # Juntamos los df de ratings y películas

In [20]:
movie_rating.head()

,movie_id,titulo,year,genero,user_id,rating
0,1,Toy Story,(1995),Animation|Children's|Comedy,1,5
1,1,Toy Story,(1995),Animation|Children's|Comedy,6,4
2,1,Toy Story,(1995),Animation|Children's|Comedy,8,4
3,1,Toy Story,(1995),Animation|Children's|Comedy,9,5
4,1,Toy Story,(1995),Animation|Children's|Comedy,10,5


In [21]:
movie_rating.shape

(1000209, 6)

Creamos un df que tiene como filas 'user_id' y columnas los titulos de películas, por lo tanto, los datos corresponden al rating que el usuario x le asigna a la película y

In [22]:
datos_completos = movie_rating.pivot_table(index=['user_id'],columns=['titulo'],values='rating')

In [23]:
datos_completos.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
datos_completos.shape

(6040, 3664)

In [25]:
datos_norm = datos_completos.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [26]:
datos_norm.fillna(0, inplace=True)

In [27]:
datos_norm.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
gc.collect()

66

In [29]:
matrix = datos_norm.values # Creamos una matriz con los valores del df normalizado

In [30]:
matrix.shape

(6040, 3664)

In [31]:
movie_similar = cosine_similarity(matrix.T) #Calculamos la distancia del coseno entre películas

In [32]:
#Creamos un df para hacer más fácil la visualización de la simulitud de las películas
movie_sim_df = pd.DataFrame(movie_similar, index = datos_norm.T.index, columns = datos_norm.T.index)

In [33]:
movie_sim_df.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
titulo,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck",1.000000,0.025770,0.016318,0.018306,-0.019676,0.00000,-0.037180,0.056074,-0.104969,0.044736,...,0.001042,0.003189,0.000000,-0.012496,0.0,0.007067,-0.017947,0.00000,0.045432,-0.018589
'Night Mother,0.025770,1.000000,-0.024688,-0.016331,-0.011163,0.00000,0.010908,-0.011531,-0.005168,-0.021604,...,0.015304,-0.016179,-0.038284,-0.012266,0.0,-0.122968,-0.000018,0.00000,0.000000,-0.000934
'Til There Was You,0.016318,-0.024688,1.000000,0.103284,0.000343,0.03036,-0.026264,0.019714,-0.037693,0.023829,...,-0.009840,0.017415,0.000000,0.050748,0.0,0.006308,-0.035657,0.00000,0.002501,0.006550
"'burbs, The",0.018306,-0.016331,0.103284,1.000000,-0.053126,0.00000,0.010561,0.017880,-0.099406,0.020187,...,0.019347,0.019538,0.014360,0.016458,0.0,-0.005947,-0.050011,0.00000,-0.009886,0.004365
...And Justice for All,-0.019676,-0.011163,0.000343,-0.053126,1.000000,0.00000,0.002182,-0.021432,0.068903,-0.003726,...,-0.056169,-0.006018,0.002910,-0.002081,0.0,-0.104608,0.010751,-0.02819,0.000000,-0.029380


In [34]:
gc.collect()

132

In [36]:
torch.cuda.empty_cache()

In [35]:
#hacemos el mismo procedimiento para el df de usuario y ver cuales son más similares entre si
usuarios_similar = cosine_similarity(matrix)

In [36]:
usuarios_sim_df = pd.DataFrame(usuarios_similar, index = datos_norm.index, columns = datos_norm.index)

In [37]:
usuarios_sim_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.032665,-0.032267,0.016359,-0.016774,0.045229,0.014314,0.013465,0.061905,-0.002180,...,0.012852,0.034246,0.047814,-0.003729,0.069408,-0.038577,0.000124,0.000000,0.011725,0.005210
2,0.032665,1.000000,0.023592,-0.016699,-0.032420,-0.019154,0.062835,-0.001777,0.060978,-0.004647,...,-0.002849,0.012514,0.080611,-0.006570,0.076127,0.089079,0.054582,-0.013096,0.032720,-0.006562
3,-0.032267,0.023592,1.000000,0.042521,-0.031153,-0.010965,0.057895,-0.040328,-0.021620,0.003058,...,-0.016524,0.005764,-0.027753,0.000000,0.009517,0.028703,0.001214,0.046408,0.048975,-0.041902
4,0.016359,-0.016699,0.042521,1.000000,-0.002730,0.032028,-0.031510,0.031002,0.021538,-0.038338,...,-0.038547,0.002558,-0.004579,0.000000,-0.051202,0.043796,0.015004,-0.073011,-0.011580,0.035714
5,-0.016774,-0.032420,-0.031153,-0.002730,1.000000,-0.045909,0.009393,0.045783,0.015280,-0.004948,...,0.019115,0.013596,-0.022016,0.011270,0.061089,0.049487,-0.024233,-0.035286,0.017825,0.064367


In [38]:
# Toma las 5 películas más 'parecidas' a la pelicula que se le de como input. También toma el score de similitud

def top_movies(movie):
    movie_sim = movie_sim_df.sort_values(by = movie, ascending=False).index[1:6]
    valores_sim = movie_sim_df.sort_values(by = movie, ascending=False).loc[:,movie].tolist()[1:6]
    zipped = zip(movie_sim, valores_sim,)
    print('Películas similares a ', movie)
    for mov, sim in zipped:
        print(mov, 'tiene similitud de ', sim)

In [39]:
# Toma los usuarios con mayor similitud dado un usuario 'x'. También toma el score de similitud

def top_users(usuario):
    if usuario not in datos_norm.index:
        return('No existe usuario')
    else:
        valores_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).loc[:,usuario].tolist()[1:6]
        usuario_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).index[1:6]
        zipped = zip(usuario_sim, valores_sim,)
        for user, sim in zipped:
            print('El usuario ', user, 'tiene similitud de ', sim) 

In [40]:
#Predice la calificación que cierto usuario le daría a alguna película

def pred_rating(movie, usuario):
    valores_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).loc[:,usuario].tolist()[1:1000]
    usuario_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).index[1:1000]
    ratings = []
    pesos = []
    
    for i, j in enumerate(usuario_sim):
        rating = datos_completos.loc[j, movie]
        similares = usuario_sim[i]
        if np.isnan(rating):
            continue
        elif not np.isnan(rating):
            ratings.append(rating*similares)
            pesos.append(similares)
    return sum(ratings)/sum(pesos)

In [42]:
top_movies('10 Things I Hate About You')

Películas similares a  10 Things I Hate About You
Cruel Intentions tiene similitud de  0.12211650014571397
Can't Hardly Wait tiene similitud de  0.11251170678798358
Never Been Kissed tiene similitud de  0.10464539011806241
She's All That tiene similitud de  0.10350514871463565
Varsity Blues tiene similitud de  0.10341563613553283


In [43]:
top_movies('Toy Story')

Películas similares a  Toy Story
Toy Story 2 tiene similitud de  0.48582011599016317
Raiders of the Lost Ark tiene similitud de  0.29344814032201527
Bug's Life, A tiene similitud de  0.28683011547309467
Shawshank Redemption, The tiene similitud de  0.2713600398538516
Back to the Future tiene similitud de  0.26793720317508835


In [44]:
top_users(2010)

El usuario  302 tiene similitud de  0.3715315991191138
El usuario  5367 tiene similitud de  0.35066281941550237
El usuario  710 tiene similitud de  0.3348417673372222
El usuario  1613 tiene similitud de  0.33410102981488193
El usuario  3389 tiene similitud de  0.33384164502656943


In [45]:
pred_rating('Toy Story', 15)

4.107177106231184